# Hydrated tweets cleaning

jsonl -> csv -> cleaned csv

dataset source: https://github.com/echen102/COVID-19-TweetIDs

In [1]:
import pandas as pd
import json
import os
import re
import collections
from datetime import datetime
import random
import tqdm

random.seed(42)

In [2]:
OUT_DIR = "../sampled_data/"
JSON_DIR = OUT_DIR + "json/"
CLEAN_DIR = OUT_DIR + "clean_2000/"

In [3]:
# for file_name in sorted(os.listdir(JSON_DIR)):
#     print(file_name)

01_sampled.jsonl
02_sampled.jsonl
03_sampled.jsonl


In [49]:
file_name = "03_sampled.jsonl"
print("processing:", file_name)

processing: 03_sampled.jsonl


In [36]:
def get_date(string):
    try:
        return datetime.strftime(datetime.strptime(string,'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d')
    except:
        print(string)
        return None

In [37]:
def process(line):
    # from json to single line df
    df_inter = pd.DataFrame([line])
    df_inter.columns = ['json_element']
    df_inter['json_element'].apply(json.loads)
    df_inter = pd.json_normalize(df_inter['json_element'].apply(json.loads))
    # extract needed columns
    df_needed = df_inter[['created_at', 'id_str', 'full_text', 'lang']]
    # format date
    df_needed["date"] = df_needed["created_at"].apply(get_date)
    df_needed.drop(["created_at"], axis=1)
    
    return df_needed

In [44]:
df = pd.DataFrame()

with tqdm.tqdm(total=os.path.getsize(JSON_DIR + file_name)) as pbar:
    with open(JSON_DIR + file_name) as reader:
        for line in reader:
            pbar.update(len(line))
            df_line = process(line.strip("\n"))
            if df_line["lang"][0] == "en":
                df = df.append(df_line)

  0%|          | 0/431104627 [00:00<?, ?it/s]/home/cwh/anaconda2/envs/p36workshop/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
 20%|█▉        | 84926569/431104627 [03:15<13:17, 434168.16it/s]


KeyboardInterrupt: 

In [46]:
print("SORT by tweets")
print(df["date"].value_counts())

2020-01-22    8274
2020-01-23    6291
Name: date, dtype: int64

In [47]:
print("SORT by date")
print(df["date"].value_counts().sort_index())

2020-01-22    8274
2020-01-23    6291
Name: date, dtype: int64


In [48]:
# Sample 2000 EN tweets per day
PER_DAY = 2000

dates = df["date"].unique()
df_2000 = pd.concat([df[df['date']==d].sample(n=PER_DAY, random_state=42) for d in dates])
df_2000["date"].value_counts()

2020-01-23    2000
2020-01-22    2000
Name: date, dtype: int64

In [ ]:
df_2000.to_csv(CLEAN_DIR + file_name.split(".")[0] + "_clean.csv", encoding='utf-8')